In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from lvmdrp.scripts import preproc


FAS_NAME = "dummy-desi-fiberassing.fits.gz"

import re


type2fmt = {
    "bool": "L",
    "binary": "L",
    "logical": "L",
    "byte": "X",
    "char": "A",
    "int16": "I",
    "int32": "J",
    "int64": "K",
    "float32": "E",
    "float64": "D"
}
type2dmy = dict(zip(type2fmt.keys(),[True, True, True, 0, "x", -9999, -9999, -9999, -9999.9, -9999.9]))

pattern = re.compile(r"\[(\d+)\]")

In [2]:
import pandas as pd

fas_data_url = "https://desidatamodel.readthedocs.io/en/latest/DESI_TARGET/fiberassign/tile-TILEID-FIELDNUM.html"

desimodel_fas_tables = []
for table in pd.read_html(fas_data_url):
    clean_table = table.fillna(-9999)
    clean_table = clean_table.applymap(lambda s: s.replace("’","").replace("‘","").replace("…","") if type(s)==str else s)
    clean_table = clean_table.applymap(lambda s: pd.to_numeric(s, errors="ignore", downcast="integer"))
    desimodel_fas_tables.append(clean_table)

In [3]:
desimodel_fas_tables[1]

,Number,EXTNAME,Type,Contents
0,HDU0,PRIMARY,IMAGE,Blank
1,HDU1,FIBERASSIGN,BINTABLE,Target assignments for each fiber
2,HDU2,SKY_MONITOR,BINTABLE,Description needed
3,HDU3,TARGETS,BINTABLE,Target catalog row-matched to FIBERASSIGN table
4,HDU4,POTENTIAL_ASSIGNMENTS,BINTABLE,All targets that could have been assigned
5,HDU5,FASSIGN,BINTABLE,Description needed
6,HDU6,FTARGETS,BINTABLE,Description needed
7,HDU7,FAVAIL,BINTABLE,Description needed


In [4]:
desimodel_fas_tables[6]

,Name,Type,Units,Description
0,FIBER,int32,-9999.0,-9999.0
1,LOCATION,int32,-9999.0,-9999.0
2,NUMTARGET,int16,-9999.0,-9999.0
3,TARGETID,int64,-9999.0,-9999.0
4,BRICKID,int32,-9999.0,-9999.0
5,BRICK_OBJID,int32,-9999.0,-9999.0
6,FA_TARGET,int64,-9999.0,-9999.0
7,FA_TYPE,binary,-9999.0,-9999.0
8,TARGET_RA,float64,-9999.0,-9999.0
9,TARGET_DEC,float64,-9999.0,-9999.0


In [5]:
dummy = fits.HDUList()

hdu = fits.PrimaryHDU(header=fits.Header(cards=map(tuple, desimodel_fas_tables[2].drop(columns="Type").values.tolist())))
hdu.name = desimodel_fas_tables[1].loc[0,"EXTNAME"]
dummy.append(hdu)

for k, j in enumerate(range(3, len(desimodel_fas_tables)-1, 2)):

    header = fits.Header(cards=map(tuple, desimodel_fas_tables[j].drop(columns="Type").values.tolist()))
    columns = []
    for i, row in desimodel_fas_tables[j+1].iterrows():

        shape = re.findall(pattern, row.Type)
        ctype = re.sub(pattern, "", row.Type)
        if len(shape) != 0:
            fmt = "%s%s"%(shape[0],type2fmt[ctype])
            arr = int(shape[0])*[type2dmy[ctype]] if ctype!="char" else [int(shape[0])*type2dmy[ctype]]
        else:
            fmt = type2fmt[ctype]
            arr = [type2dmy[ctype]]

        columns.append(fits.Column(name=row.Name, format=fmt, array=arr, unit=row.Units if row.Units!=-9999 else None))

    hdu = fits.BinTableHDU.from_columns(header=header, columns=columns)
    hdu.name = desimodel_fas_tables[1].loc[k+1,"EXTNAME"]
    dummy.append(hdu)

dummy.writeto(FAS_NAME, overwrite=True)
dummy.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   ()      
  1  FIBERASSIGN    1 BinTableHDU    298   5R x 127C   ['K', 'I', 'J', 'J', 'J', 'J', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'K', 'X', '3A', 'E', 'E', 'I', 'J', 'D', 'J', 'J', 'I', 'J', '8A', 'J', '4A', 'E', 'E', '5E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'I', 'I', 'I', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'X', 'X', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '2A', 'K', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'K', 'E', 'L', 'E', 'E', 'I', 'E', '1A', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'D', 'D', 'D']   
  2  SKY_MONITOR    1 BinTableHDU     70   1R x 24C   ['J', 'J', 'I', 'K', 'J', 'J', 'K', 'L', 'D', 'D', 'E', 'E', '8A', 'J', 'I', 'J', 'J